In [1]:
from pathlib import Path
import numpy as np
import matplotlib as plt
import k3d
import trimesh
import torch
import skimage

In [2]:
torch.cuda.is_available()

True

In [3]:
from model.deepsdf import DeepSDFDecoder
from util.model import summarize_model
from training import train_deepsdf
from data.shape_implicit import ShapeImplicit
from util.visualization import visualize_mesh, visualize_pointcloud

In [4]:

deepsdf = DeepSDFDecoder(latent_size=256)
print(summarize_model(deepsdf))

# input to the network is a concatenation of point coordinates (3) and the latent code (256 in this example);
# here we use a batch of 4096 points
input_tensor = torch.randn(4096, 3 + 256)
predictions = deepsdf(input_tensor)

print('\nOutput tensor shape: ', predictions.shape)  # expected output: 4096, 1

num_trainable_params = sum(p.numel() for p in deepsdf.parameters() if p.requires_grad) / 1e6
print(f'\nNumber of traininable params: {num_trainable_params:.2f}M')  # expected output: ~1.8M

   | Name      | Type           | Params 
-----------------------------------------------
0  | layer1    | Sequential     | 790010 
1  | layer1.0  | Linear         | 133632 
2  | layer1.1  | Dropout        | 0      
3  | layer1.2  | ReLU           | 0      
4  | layer1.3  | Linear         | 263168 
5  | layer1.4  | Dropout        | 0      
6  | layer1.5  | ReLU           | 0      
7  | layer1.6  | Linear         | 263168 
8  | layer1.7  | Dropout        | 0      
9  | layer1.8  | ReLU           | 0      
10 | layer1.9  | Linear         | 130042 
11 | layer1.10 | Dropout        | 0      
12 | layer1.11 | ReLU           | 0      
13 | layer2    | Sequential     | 1053186
14 | layer2.0  | Linear         | 263168 
15 | layer2.1  | Dropout        | 0      
16 | layer2.2  | ReLU           | 0      
17 | layer2.3  | Linear         | 263168 
18 | layer2.4  | Dropout        | 0      
19 | layer2.5  | ReLU           | 0      
20 | layer2.6  | Linear         | 263168 
21 | layer2.7  | Dropout    

In [25]:


overfit_config = {
    'experiment_name': '3_2_deepsdf_overfit',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': True,
    'num_sample_points': 40000,
    'latent_code_length': 256,
    'batch_size': 16,
    'resume_ckpt': None,
    'learning_rate_model': 0.0005,
    'learning_rate_code': 0.001,
    'lambda_code_regularization': 0.0001,
    'max_epochs': 2000,
    'print_every_n': 50,
    'visualize_every_n': 250,
}

train_deepsdf.main(overfit_config)  # expected loss around 0.0062

Using device: cuda:0
[049/00000] train_loss: 0.035846
[099/00000] train_loss: 0.023321
[149/00000] train_loss: 0.016326
[199/00000] train_loss: 0.012821
[249/00000] train_loss: 0.011374
[299/00000] train_loss: 0.010310
[349/00000] train_loss: 0.009544
[399/00000] train_loss: 0.009031
[449/00000] train_loss: 0.008525
[499/00000] train_loss: 0.008322
[549/00000] train_loss: 0.007706
[599/00000] train_loss: 0.007494
[649/00000] train_loss: 0.007348
[699/00000] train_loss: 0.007253
[749/00000] train_loss: 0.007096
[799/00000] train_loss: 0.006998
[849/00000] train_loss: 0.006862
[899/00000] train_loss: 0.006766
[949/00000] train_loss: 0.006676
[999/00000] train_loss: 0.006567
[1049/00000] train_loss: 0.006429
[1099/00000] train_loss: 0.006371
[1149/00000] train_loss: 0.006317
[1199/00000] train_loss: 0.006258
[1249/00000] train_loss: 0.006216
[1299/00000] train_loss: 0.006166
[1349/00000] train_loss: 0.006113
[1399/00000] train_loss: 0.006058
[1449/00000] train_loss: 0.006025
[1499/00000] 

In [27]:
# Load and visualize GT mesh of the overfit sample
gt_mesh = ShapeImplicit.get_mesh('7e728818848f191bee7d178666aae23d')
print('GT')
visualize_mesh(gt_mesh.vertices, gt_mesh.faces, flip_axes=True)

# Load and visualize reconstructed overfit sample; it's okay if they don't look visually exact, since we don't run 
# the training too long and have a learning rate decay while training 
mesh_path = "runs/3_2_deepsdf_overfit/meshes/01999_000.obj"
overfit_output = trimesh.load(mesh_path)
print('Overfit')
visualize_mesh(overfit_output.vertices, overfit_output.faces, flip_axes=True)

GT


Output()

Overfit


Output()

In [6]:
from training import train_deepsdf

generalization_config = {
    'experiment_name': '3_2_deepsdf_generalization',
    'device': 'cuda:0',  # run this on a gpu for a reasonable training time
    'is_overfit': False,
    'num_sample_points': 4096, # you can adjust this such that the model fits on your gpu
    'latent_code_length': 256,
    'batch_size': 1,
    'resume_ckpt': None,
    'learning_rate_model': 0.0005,
    'learning_rate_code': 0.001,
    'lambda_code_regularization': 0.0001,
    'max_epochs': 2000,  # not necessary to run for 2000 epochs if you're short on time, at 500 epochs you should start to see reasonable results
    'print_every_n': 50,
    'visualize_every_n': 5000,
}

train_deepsdf.main(generalization_config)

Using device: cuda:0
[000/00049] train_loss: 0.038442
[000/00099] train_loss: 0.035935
[000/00149] train_loss: 0.033375
[000/00199] train_loss: 0.033340
[000/00249] train_loss: 0.032919
[000/00299] train_loss: 0.032582
[000/00349] train_loss: 0.033365
[000/00399] train_loss: 0.031029
[000/00449] train_loss: 0.032137
[000/00499] train_loss: 0.030671
[000/00549] train_loss: 0.031336
[000/00599] train_loss: 0.032829
[000/00649] train_loss: 0.031796
[000/00699] train_loss: 0.032844
[000/00749] train_loss: 0.031459
[000/00799] train_loss: 0.031331
[000/00849] train_loss: 0.032159
[000/00899] train_loss: 0.030964
[000/00949] train_loss: 0.032007
[000/00999] train_loss: 0.033122
[000/01049] train_loss: 0.031459
[000/01099] train_loss: 0.031270
[000/01149] train_loss: 0.032139
[000/01199] train_loss: 0.031048
[001/00023] train_loss: 0.031830
[001/00073] train_loss: 0.032459
[001/00123] train_loss: 0.031237
[001/00173] train_loss: 0.030809
[001/00223] train_loss: 0.030694
[001/00273] train_loss

KeyboardInterrupt: 

In [5]:
from inference.infer_deepsdf import InferenceHandlerDeepSDF

device = torch.device('cuda:0') 

inference_handler = InferenceHandlerDeepSDF(256, "runs/3_2_deepsdf_generalization", device)

In [6]:
points, sdf = ShapeImplicit.get_all_sdf_samples("b351e06f5826444c19fb4103277a6b93")

inside_points = points[sdf[:, 0] < 0, :].numpy()
outside_points = points[sdf[:, 0] > 0, :].numpy()

# visualize observed points; you'll observe that the observations are very complete
print('Observations with negative SDF (inside)')
visualize_pointcloud(inside_points, 0.025, flip_axes=True)
print('Observations with positive SDF (outside)')
visualize_pointcloud(outside_points, 0.025, flip_axes=True)

Observations with negative SDF (inside)


Output()

Observations with positive SDF (outside)


Output()

In [7]:
# reconstruct
vertices, faces = inference_handler.reconstruct(points, sdf, 800)
# visualize
visualize_mesh(vertices, faces, flip_axes=True)

[00000] optim_loss: 0.031894
[00050] optim_loss: 0.008029
[00100] optim_loss: 0.006347
[00150] optim_loss: 0.005890
[00200] optim_loss: 0.005879
[00250] optim_loss: 0.005604
[00300] optim_loss: 0.005261
[00350] optim_loss: 0.005147
[00400] optim_loss: 0.005209
[00450] optim_loss: 0.005113
[00500] optim_loss: 0.004998
[00550] optim_loss: 0.004825
[00600] optim_loss: 0.004784
[00650] optim_loss: 0.004640
[00700] optim_loss: 0.004755
[00750] optim_loss: 0.004710
Optimization complete.


Output()

In [8]:
# get observed data
points, sdf = ShapeImplicit.get_all_sdf_samples("b351e06f5826444c19fb4103277a6b93_incomplete")

inside_points = points[sdf[:, 0] < 0, :].numpy()
outside_points = points[sdf[:, 0] > 0, :].numpy()

# visualize observed points; you'll observe that the observations are incomplete
# making this is a shape completion task
print('Observations with negative SDF (inside)')
visualize_pointcloud(inside_points, 0.025, flip_axes=True)
print('Observations with positive SDF (outside)')
visualize_pointcloud(outside_points, 0.025, flip_axes=True)

Observations with negative SDF (inside)


Output()

Observations with positive SDF (outside)


Output()

In [9]:
# reconstruct
vertices, faces = inference_handler.reconstruct(points, sdf, 800)
# visualize
visualize_mesh(vertices, faces, flip_axes=True)

[00000] optim_loss: 0.032641
[00050] optim_loss: 0.008166
[00100] optim_loss: 0.005987
[00150] optim_loss: 0.005718
[00200] optim_loss: 0.005049
[00250] optim_loss: 0.004744
[00300] optim_loss: 0.004648
[00350] optim_loss: 0.004559
[00400] optim_loss: 0.004415
[00450] optim_loss: 0.004512
[00500] optim_loss: 0.004287
[00550] optim_loss: 0.004618
[00600] optim_loss: 0.004271
[00650] optim_loss: 0.004155
[00700] optim_loss: 0.004162
[00750] optim_loss: 0.004096
Optimization complete.


Output()

In [10]:
from data.shape_implicit import ShapeImplicit
from util.visualization import visualize_mesh

mesh = ShapeImplicit.get_mesh("494fe53da65650b8c358765b76c296")
print('GT Shape A')
visualize_mesh(mesh.vertices, mesh.faces, flip_axes=True)

mesh = ShapeImplicit.get_mesh("5ca1ef55ff5f68501921e7a85cf9da35")
print('GT Shape B')
visualize_mesh(mesh.vertices, mesh.faces, flip_axes=True)

GT Shape A


C:\Users\baris\anaconda3\envs\adl4cv\lib\site-packages\traittypes\traittypes.py:97: UserWarning: Given trait value dtype "uint32" does not match required type "uint32". A coerced copy has been created.
  warnings.warn(


Output()

GT Shape B


Output()

In [13]:
from inference.infer_deepsdf import InferenceHandlerDeepSDF

inference_handler = InferenceHandlerDeepSDF(256, "runs/3_2_deepsdf_generalization", torch.device('cuda:0'))
# interpolate; also exports interpolated meshes to disk
inference_handler.interpolate('494fe53da65650b8c358765b76c296', '5ca1ef55ff5f68501921e7a85cf9da35', 60)

In [14]:
from util.mesh_collection_to_gif import  meshes_to_gif
from util.misc import show_gif

# create list of meshes (just exported) to be visualized
mesh_paths = sorted([x for x in Path("/runs/3_2_deepsdf_generalization/interpolation").iterdir() if int(x.name.split('.')[0].split("_")[1]) == 0], key=lambda x: int(x.name.split('.')[0].split("_")[0]))
mesh_paths = mesh_paths + mesh_paths[::-1]

# create a visualization of the interpolation process
meshes_to_gif(mesh_paths, "exercise_3/runs/3_2_deepsdf_generalization/latent_interp.gif", 20)
show_gif("exercise_3/runs/3_2_deepsdf_generalization/latent_interp.gif")

ModuleNotFoundError: No module named 'pyrender'